In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import plotly.express as px
import os
from datetime import datetime
import time
import plotly.graph_objects as go
import pickle


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)


In [2]:
all_feature_list = [folder for folder in os.listdir(".") if "." not in folder]
# all_feature_list = [feature for feature in all_feature_list if "_60min" in feature]
# all_feature_list = [feature for feature in all_feature_list if feature.split("_")[0] not in ['AROON','RSI','STCO']]
all_feature_list

['AROON_120min',
 'AROON_240min',
 'AROON_360min',
 'AROON_480min',
 'AROON_600min',
 'AROON_60min',
 'AROON_90min',
 'BB_120min',
 'BB_240min',
 'BB_360min',
 'BB_480min',
 'BB_600min',
 'BB_60min',
 'BB_90min',
 'EMA_120min',
 'EMA_240min',
 'EMA_360min',
 'EMA_480min',
 'EMA_600min',
 'EMA_60min',
 'EMA_90min',
 'High_120min',
 'High_240min',
 'High_360min',
 'High_480min',
 'High_600min',
 'High_60min',
 'High_90min',
 'Low_120min',
 'Low_240min',
 'Low_360min',
 'Low_480min',
 'Low_600min',
 'Low_60min',
 'Low_90min',
 'MACD_120min',
 'MACD_240min',
 'MACD_360min',
 'MACD_480min',
 'MACD_600min',
 'MACD_60min',
 'MACD_90min',
 'MA_120min',
 'MA_240min',
 'MA_360min',
 'MA_480min',
 'MA_600min',
 'MA_60min',
 'MA_90min',
 'ROC_Close_120min',
 'ROC_Close_240min',
 'ROC_Close_360min',
 'ROC_Close_480min',
 'ROC_Close_600min',
 'ROC_Close_60min',
 'ROC_Close_90min',
 'RSI_120min',
 'RSI_240min',
 'RSI_360min',
 'RSI_480min',
 'RSI_600min',
 'RSI_60min',
 'RSI_90min',
 'STCO_120min',
 

In [3]:
def data_set_builder(fx_pair, years, features):
    """
    fx_pair: a given currency pair: "EURUSD", "GBPCHF", etc...
    years: if int, a single year, else, a list of years.
    features: a list of features, as described in its file structure. E.G "ROC_Low_90min"
    """
    if type(years) == int: years = [years]
    df = None
    for year in tqdm(years):
        temp_df = pd.read_csv("../Cleaned_OHLC_FOREX_Data_10_min/{}/{}_{}_10min.csv".format(fx_pair, fx_pair,  year)).drop(columns =['Unnamed: 0'])     
        
        for feature in features:
            feature_df = pd.read_csv("{}/{}/{}_{}_{}.csv".format(feature, fx_pair, feature, fx_pair, year)).drop(columns =['Unnamed: 0'])
            temp_df = pd.concat([temp_df, feature_df], axis=1)
        
            
        if type(df) == None:
            df = temp_df.copy()
        else: 
            df = pd.concat([df, temp_df], axis=0)
            
    #replace the NaNs found in these columns. The Nans exist due to the ways the formulas are created.
    for col in df:
        if "ROC" in col:
            df[col].fillna(0, inplace = True)
        elif "STCO" in col:
            df[col].fillna(0.5, inplace = True)
        elif "AROON" in col:
            df[col].fillna(0, inplace = True)
        else:
            pass
        
    df.drop(columns = ['Open','High','Low','Close'], inplace = True)
    return df


clean_df = data_set_builder('EURUSD', [i for i in range(2012,2020)], all_feature_list)
clean_df

clean_df.isna().sum()

Date_time                0
AROON_120min_up          0
AROON_120min_down        0
AROON_240min_up          0
AROON_240min_down        0
AROON_360min_up          0
AROON_360min_down        0
AROON_480min_up          0
AROON_480min_down        0
AROON_600min_up          0
AROON_600min_down        0
AROON_60min_up           0
AROON_60min_down         0
AROON_90min_up           0
AROON_90min_down         0
BB_120min_upper          0
BB_120min_lower          0
BB_240min_upper          0
BB_240min_lower          0
BB_360min_upper          0
BB_360min_lower          0
BB_480min_upper          0
BB_480min_lower          0
BB_600min_upper          0
BB_600min_lower          0
BB_60min_upper           0
BB_60min_lower           0
BB_90min_upper           0
BB_90min_lower           0
EMA_120min               0
EMA_240min               0
EMA_360min               0
EMA_480min               0
EMA_600min               0
EMA_60min                0
EMA_90min                0
High_120min              0
H

In [4]:
def standardize_column(df, feature, fx_pair):
    """
    Use 2012-2019 as test set. That is, use these years to create standardized scalar value, to enforce a std of 
    1 on each feature column.
    
    """

    # For MACD and Bollinger bands there are more than one column. Just use the first column.
    df = df[[col for col in df if feature in col]]
    scalar_val = 1 / df.iloc[:,0].std() 
    if ("STCO" in feature) or ("RSI" in feature) or ("AROON" in feature):
        scalar_val = 2 # creating a min/max of -1 and 1.
        
    pickle_path = "{}/{}/Feature_standardized_multiplier_2012_2019.pickle".format(feature, fx_pair)
    with open(pickle_path, 'wb') as file:
        pickle.dump(scalar_val, file)
    
    for year in range(2012,2021):        
        standardized_feature_df_path = "{}/{}/{}_{}_{}_standardized.csv".format(feature, fx_pair, feature, fx_pair, year)
        feature_df = pd.read_csv("{}/{}/{}_{}_{}.csv".format(feature, fx_pair, feature, fx_pair, year)).drop(columns =['Unnamed: 0'])
        
        #replace the NaNs found in these columns. 
        # The Nans exist due to the ways the formulas are created.
        # Even when cleaned before, when loading in the data, the same entries of Nans occour.
        for col in feature_df:
            if "ROC" in col:
                feature_df[col].fillna(0, inplace = True)
            elif "STCO" in col:
                feature_df[col].fillna(0.5, inplace = True)
            elif "AROON" in col:
                feature_df[col].fillna(0, inplace = True)
            else:
                pass
        
        feature_df = feature_df * scalar_val
        if ("STCO" in feature) or ("RSI" in feature) or ("AROON" in feature):
            feature_df = feature_df - 1
            
        feature_df.fillna(0, inplace = True)    # All features have arround at 0, which also should be close to the mean.
        for feature_col in feature_df.columns:  # just because of BollingerBands...
            if feature_df[feature_col].isna().sum() != 0:
                print("amount of nans in {} {} {}: {}".format(fx_pair, year, feature_col, feature_df.isna().sum()))
        
        feature_df.to_csv(standardized_feature_df_path)
    
    
all_feature_list = [folder for folder in os.listdir(".") if "." not in folder]

for fx_pair in ['EURUSD','EURGBP','GBPUSD']:
    clean_df = data_set_builder(fx_pair, [i for i in range(2012,2020)], all_feature_list) # Only include up to 2019
    for feature in tqdm(all_feature_list):
        standardize_column(clean_df, feature, fx_pair)


In [6]:
def data_set_builder_ML_bench(fx_pair, years, features, future_candles):
    """
    fx_pair: a given currency pair: "EURUSD", "GBPCHF", etc...
    years: if int, a single year, else, a list of years.
    features: a list of features, as described in its file structure. E.G "ROC_Low_90min"
    """
    if type(years) == int: years = [years]
    df = None
    for year in tqdm(years):
        temp_df = pd.read_csv("../../Data/Cleaned_OHLC_FOREX_Data_10_min/{}/{}_{}_10min.csv".format(fx_pair, fx_pair,  year)).drop(columns =['Unnamed: 0'])     
        
        target_df = pd.read_csv("../../Data/Target_Bank/Regression/{}/{}_{}_{}_candle_future.csv".format(fx_pair,fx_pair,year,future_candles)).drop(columns =['Unnamed: 0'])
        temp_df = pd.concat([temp_df, target_df], axis=1)
        
        for feature in features:
            feature_df = pd.read_csv("../../Data/Feature_Bank/{}/{}/{}_{}_{}_standardized.csv".format(feature, fx_pair, feature, fx_pair, year)).drop(columns =['Unnamed: 0'])
            #feature_df = pd.read_csv("../Data/Feature_Bank/{}/{}/{}_{}_{}.csv".format(feature, fx_pair, feature, fx_pair, year)).drop(columns =['Unnamed: 0'])
            temp_df = pd.concat([temp_df, feature_df], axis=1)
        
            
        if type(df) == None:
            df = temp_df.copy()
        else: 
            df = pd.concat([df, temp_df], axis=0)
                    
    df.drop(columns = ['Open','High','Low','Close'], inplace = True)
    df.reset_index(inplace = True, drop = True )
    return df


clean_df = data_set_builder_ML_bench('EURUSD', [i for i in range(2012,2021)], all_feature_list, 18)

print(clean_df.isna().sum())

clean_df.head(20)



Date_time                0
18_candle_target         0
AROON_120min_up          0
AROON_120min_down        0
AROON_240min_up          0
AROON_240min_down        0
AROON_360min_up          0
AROON_360min_down        0
AROON_480min_up          0
AROON_480min_down        0
AROON_600min_up          0
AROON_600min_down        0
AROON_60min_up           0
AROON_60min_down         0
AROON_90min_up           0
AROON_90min_down         0
BB_120min_upper          0
BB_120min_lower          0
BB_240min_upper          0
BB_240min_lower          0
BB_360min_upper          0
BB_360min_lower          0
BB_480min_upper          0
BB_480min_lower          0
BB_600min_upper          0
BB_600min_lower          0
BB_60min_upper           0
BB_60min_lower           0
BB_90min_upper           0
BB_90min_lower           0
EMA_120min               0
EMA_240min               0
EMA_360min               0
EMA_480min               0
EMA_600min               0
EMA_60min                0
EMA_90min                0


,Date_time,18_candle_target,AROON_120min_up,AROON_120min_down,AROON_240min_up,AROON_240min_down,AROON_360min_up,AROON_360min_down,AROON_480min_up,AROON_480min_down,AROON_600min_up,AROON_600min_down,AROON_60min_up,AROON_60min_down,AROON_90min_up,AROON_90min_down,BB_120min_upper,BB_120min_lower,BB_240min_upper,BB_240min_lower,BB_360min_upper,BB_360min_lower,BB_480min_upper,BB_480min_lower,BB_600min_upper,BB_600min_lower,BB_60min_upper,BB_60min_lower,BB_90min_upper,BB_90min_lower,EMA_120min,EMA_240min,EMA_360min,EMA_480min,EMA_600min,EMA_60min,EMA_90min,High_120min,High_240min,High_360min,High_480min,High_600min,High_60min,High_90min,Low_120min,Low_240min,Low_360min,Low_480min,Low_600min,Low_60min,Low_90min,MACD_120min,MACD_120min_signal,MACD_120min_crossover,MACD_240min,MACD_240min_signal,MACD_240min_crossover,MACD_360min,MACD_360min_signal,MACD_360min_crossover,MACD_480min,MACD_480min_signal,MACD_480min_crossover,MACD_600min,MACD_600min_signal,MACD_600min_crossover,MACD_60min,MACD_60min_signal,MACD_60min_crossover,MACD_90min,MACD_90min_signal,MACD_90min_crossover,MA_120min,MA_240min,MA_360min,MA_480min,MA_600min,MA_60min,MA_90min,ROC_Close_120min,ROC_Close_240min,ROC_Close_360min,ROC_Close_480min,ROC_Close_600min,ROC_Close_60min,ROC_Close_90min,RSI_120min,RSI_240min,RSI_360min,RSI_480min,RSI_600min,RSI_60min,RSI_90min,STCO_120min,STCO_240min,STCO_360min,STCO_480min,STCO_600min,STCO_60min,STCO_90min
0,2012-01-02 02:00:00,15.9,0.833333,0.833333,0.916667,0.916667,0.944444,0.944444,0.958333,0.958333,0.966667,0.966667,0.666667,0.666667,0.777778,0.777778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.170609,0.122521,0.101401,0.088895,0.080365,0.239380,0.196231,-1.153737,-0.827127,-0.683997,-0.599184,-0.541895,-1.617666,-1.327062,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2012-01-02 02:10:00,11.8,1.000000,0.666667,1.000000,0.833333,1.000000,0.888889,1.000000,0.916667,1.000000,0.933333,1.000000,0.333333,1.000000,0.555556,0.013912,-0.006644,0.009917,-0.004736,0.008154,-0.003894,0.007113,-0.003397,0.006410,-0.003061,0.019542,-0.009333,0.016023,-0.007652,0.006278,0.006278,0.006278,0.006278,0.006278,0.006278,0.006278,0.217139,0.155936,0.129055,0.113139,0.102283,0.304666,0.249749,-1.145780,-0.821423,-0.679280,-0.595052,-0.538158,-1.606510,-1.317910,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006278,0.006278,0.006278,0.006278,0.006278,0.006278,0.006278,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
2,2012-01-02 02:20:00,4.5,0.833333,0.500000,0.916667,0.750000,0.944444,0.833333,0.958333,0.875000,0.966667,0.900000,0.666667,0.000000,0.777778,0.333333,0.087027,-0.324440,0.062035,-0.231268,0.051006,-0.190153,0.044498,-0.165888,0.040099,-0.149489,0.122244,-0.455730,0.100231,-0.373664,-0.205070,-0.205070,-0.205070,-0.205070,-0.205070,-0.205070,-0.205070,0.023265,0.016707,0.013827,0.012122,0.010959,0.032643,0.026759,-1.344700,-0.964031,-0.797210,-0.698359,-0.631588,-1.885418,-1.546713,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.221199,0.147466,0.073733,0.000000,0.000000,0.000000,-0.205070,-0.205070,-0.205070,-0.205070,-0.205070,-0.205

In [7]:
with open("MACD_120min/EURUSD/Feature_standardized_multiplier_2012_2019.pickle", 'rb') as pickle_file:
    multiplier = pickle.load(pickle_file)

print("multiplier: ", multiplier)
print("Standard deviation: ", 1/multiplier)

multiplier:  2335.701017832541
Standard deviation:  0.0004281369885808285
